<a href="https://colab.research.google.com/github/bascoul/TP_Deep_Learning/blob/master/text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Authors.



In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Charger le texte

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/text"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/load_data/text.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/load_data/text.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/load_data/text.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Ce tutoriel fournit un exemple d'utilisation de `tf.data.TextLineDataset` pour charger des exemples à partir de fichiers texte. Le fichier `TextLineDataset` est conçu pour créer un ensemble de données à partir d'un fichier texte, dans lequel chaque exemple est une ligne de texte du fichier original. Ceci est potentiellement utile pour toutes les données textuelles qui sont principalement basées sur des lignes (par exemple, la poésie ou les journaux d'erreurs).

Dans ce tutoriel, nous utiliserons trois traductions anglaises différentes du même ouvrage, l'Illiade d'Homère, et nous formerons un modèle pour identifier le traducteur à qui une seule ligne de texte a été donnée.

## Setup

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf

import tensorflow_datasets as tfds
import os

     |████████████████████████████████| 533.1MB 24kB/s 
     |████████████████████████████████| 460kB 48.9MB/s 
     |████████████████████████████████| 2.9MB 47.9MB/s 
     |████████████████████████████████| 2.8MB 51.7MB/s 
     |████████████████████████████████| 778kB 46.4MB/s 
ERROR: tensorflow 1.15.0 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 1.15.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
  Found existing installation: gast 0.2.2
    Uninstalling gast-0.2.2:
      Successfully uninstalled gast-0.2.2
  Found existing installation: h5py 2.8.0
    Uninstalling h5py-2.8.0:
      Successfully uninstalled h5py-2.8.0


Les textes des trois traductions sont de :

 - [William Cowper](https://en.wikipedia.org/wiki/William_Cowper) — [text](https://storage.googleapis.com/download.tensorflow.org/data/illiad/cowper.txt)

Achilles sing, O Goddess! Peleus' son;

His wrath pernicious, who ten thousand woes

Caused to Achaia's host, sent many a soul

Illustrious into Ades premature,

..... en tout 19143 lignes


 - [Edward, Earl of Derby](https://en.wikipedia.org/wiki/Edward_Smith-Stanley,_14th_Earl_of_Derby) — [text](https://storage.googleapis.com/download.tensorflow.org/data/illiad/derby.txt)

Of Peleus' son, Achilles, sing, O Muse,

The vengeance, deep and deadly; whence to Greece

Unnumbered ills arose; which many a soul

Of mighty warriors to the viewless shades

..... en tout 18334 lignes

- [Samuel Butler](https://en.wikipedia.org/wiki/Samuel_Butler_%28novelist%29) — [text](https://storage.googleapis.com/download.tensorflow.org/data/illiad/butler.txt)

Sing, O goddess, the anger of Achilles son of Peleus, that brought

countless ills upon the Achaeans. Many a brave soul did it send

hurrying down to Hades, and many a hero did it yield a prey to dogs and

vultures, for so were the counsels of Jove fulfilled from the day on

.... en tout 12131 lignes

Les fichiers texte utilisés dans ce tutoriel ont subi quelques tâches de prétraitement typiques, la plupart du temps en supprimant des éléments - en-tête et pied de page du document, numéros de ligne, titres de chapitre. Téléchargez ces fichiers légèrement modifiés en local.

In [0]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
  text_dir = tf.keras.utils.get_file(name, origin=DIRECTORY_URL+name)
  
parent_dir = os.path.dirname(text_dir)

parent_dir

811008/807992 [==============================] - 0s 0us/step


'/root/.keras/datasets'

## Charger le texte dans les ensembles de données

Itérer les fichiers, en chargeant chacun d'entre eux dans son propre ensemble de données.

Chaque exemple doit être étiqueté individuellement, donc utilisez `tf.data.Dataset.map` pour appliquer une fonction d'étiquetage à chacun. Cela va itérer sur chaque exemple de l'ensemble de données, en renvoyant des paires (`exemple, label`).

In [0]:
def labeler(example, index):
  # Rappel, la fonction tf.cast permet de forcer le type d'index à un entier sur 64 bits
  return example, tf.cast(index, tf.int64)  

labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
  # Récupérer l'ensemble des lignes de file_name dans un objet lines_dataset de type tf.data.Dataset
  lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
  # Créer un mappage entre une ligne de line_dataset et le numéro de la traduction i
  labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
  # Ajouter labeled_dataset à labeled_data_sets pour n'obtenir en sortie qu'un seul tableau contenat les trois listes
  labeled_data_sets.append(labeled_dataset)

Combinez ces ensembles de données étiquetés en un seul ensemble de données, et mélangez le tout.


In [0]:
# Taille du buffer utilisé lors du mélange des lignes, il est supérieur au nombre de lignes
# afin de toutes les mélanger
BUFFER_SIZE = 50000
# Taille du lot à traiter qui correspond dans notre cas à fixer la longueur des lignes
# Les lignes plus courtes seront complétées par des zéros
BATCH_SIZE = 64
# Nombre d'éléments de l'ensemble de test
TAKE_SIZE = 5000

In [0]:
# Concatener les trois listes pour ne plus avoir dans all_labeled_data qu'une seule liste
# contenant toutes les lignes des trois textes
all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)
# Mélanger les lignes dans la liste avec la fonction shuffle de l'objet tf.data.Dataset
# le BUFFER_SIZE de 50000 permet de s'assurer que les 49608 lignes seront bien toutes mélangées
all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

On peut utiliser `tf.data.Dataset.take` et `print` pour voir que les paires sont bien du type `(example, label)`. Les propriétés `numpy` montrent les valeurs de chaques Tenseurs.

In [0]:
for ex in all_labeled_data.take(5):
  print(ex)

(<tf.Tensor: shape=(), dtype=string, numpy=b'Olympus, and with such impetuous speed?'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'The son of Tydeus was in two minds whether or no to turn his horses'>, <tf.Tensor: shape=(), dtype=int64, numpy=2>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'Thus was the head of Hector being dishonoured in the dust. His mother'>, <tf.Tensor: shape=(), dtype=int64, numpy=2>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'Prevailing, lured him with the bait of love.'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'He said, and furious with his spear again'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


## Encoder les lignes de texte avec des nombres

Les modèles d'apprentissage machine fonctionnent sur des nombres, et non sur des mots, de sorte que les valeurs des chaînes doivent être converties en listes de nombres. Pour ce faire, il faut faire correspondre chaque mot unique à un nombre entier unique.

### Construire le vocabulaire

Tout d'abord, créez un vocabulaire en transformant le texte en une collection de mots uniques. Il y a plusieurs façons de faire cela dans TensorFlow et Python. Pour ce tutoriel :

1. Itérer sur la valeur "numpy" de chaque exemple.
2. Utilisez `tfds.features.text.Tokenizer` pour la diviser en tokens (mots).
3. Rassemblez ces tokens dans un ensemble Python, pour supprimer les doublons.
4. Obtenez la taille du vocabulaire pour une utilisation ultérieure.

In [0]:
# Le Tokenizer va nous permettre de rechercher les blocs de caractères (mots) dans les lignes
tokenizer = tfds.features.text.Tokenizer()
# Créer un ensemble (set)
vocabulary_set = set()
# Récupérer les lignes une à une dans all_labeled_data
for text_tensor, _ in all_labeled_data:
  # Dans chaque ligne, les tokens sont stockés dans some_tokens
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  # Avec la commande update sur un ensemble, on n'ajoute que les éléments qui n'étaient pas dans celle-ci 
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

17178

### Encoder les exemples

Créez un encodeur en passant le "jeu de vocabulaire" à `tfds.features.text.TokenTextEncoder`. La méthode `encode` de l'encodeur prend une chaîne de texte et retourne une liste d'entiers.

In [0]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

Vous pouvez l'essayer sur une seule ligne pour voir à quoi ressemble le résultat.

In [0]:
example_text = next(iter(all_labeled_data))[0].numpy()
print(example_text)

b'Olympus, and with such impetuous speed?'


In [0]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

[15371, 603, 9411, 6502, 4409, 10445]


Maintenant, lancez l'encodeur sur le jeu de données en créant une fonction encode qui prend la ligne et son label en entrée et la ligne encodée et son label (qui ne change pas) en sortie.

In [0]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

Vous voulez utiliser `Dataset.map` pour appliquer cette fonction à chaque élément de l'ensemble de données.  Le fichier `Dataset.map` fonctionne en mode graphe.

* Les tenseurs du graphe n'ont pas de valeur. 
* En mode graphe, vous ne pouvez utiliser que les opérations et les fonctions TensorFlow. 

Vous ne pouvez donc pas `.map` cette fonction directement : Vous devez l'intégrer dans une `tf.py_function`. La fonction `tf.py_function` passera les tensors réguliers (avec une valeur et une méthode `.numpy()` pour y accéder), à la fonction python enveloppée.

In [0]:
def encode_map_fn(text, label):
  # py_function ne définit pas la forme des tenseurs retournés.
  # tf.py_function prend en paramètres, le nom de la fonction, les entrées de celle-ci
  # et le type des sorties au format tensorflow, ici des entiers tf.int64
  encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.int64))

  # `tf.data.Datasets` fonctionne mieux si tous les composants ont des formes définies
  #  on doit donc définir les formes manuellement: 
  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label


all_encoded_data = all_labeled_data.map(encode_map_fn)

## Diviser l'ensemble de données en lots de test et d'apprentissage

Utilisez `tf.data.Dataset.take` et `tf.data.Dataset.skip` pour créer un petit ensemble de données de test et un ensemble d'apprentissage plus important.

Avant d'être passés dans le modèle, les ensembles de données doivent être mis en lots. En général, les exemples à l'intérieur d'un lot doivent avoir la même taille et la même forme. Mais les exemples de ces ensembles de données n'ont pas tous la même taille - chaque ligne de texte a un nombre de mots différent. Utilisez donc `tf.data.Dataset.padded_batch` (au lieu de `batch`) pour remplir les exemples avec la même taille.

In [0]:
# skip(TAKE_SIZE) implique que l'on ne garde que les (49608 - 5000) lignes pour l'apprentissage
# On remélange l'ensemble obtenu avec shuffle
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
# Toutes les lignes n'ont pas la même longueur, on remplit de réro la fin des lignes
# jusqu'à obtenir BATCH_SIZE (64) éléments
train_data = train_data.padded_batch(BATCH_SIZE, padded_shapes=([None],[]))
# take(TAKE_SIZE) permet d'obtenir un ensemble de test de 5000 lignes
test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE, padded_shapes=([None],[]))

Note : A partir de **TensorFlow 2.2**, l'argument `padded_shapes` n'est plus nécessaire. Le comportement par défaut est de capitonner tous les axes au plus long du lot.

In [0]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE)

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE)

Maintenant, `test_data` et `train_data` ne sont pas des collections de paires (`example, label`), mais des collections de lots. Chaque lot est une paire de (*many examples*, *many labels*) représentée comme des tableaux.

Pour illustrer :

In [0]:
sample_text, sample_labels = next(iter(test_data))

sample_text[0], sample_labels[0]

Depuis que nous avons introduit un nouveau codage de token (le zéro utilisé pour le padding), la taille du vocabulaire a augmenté de un.

In [0]:
vocab_size += 1

## Construire le modèle



In [0]:
model = tf.keras.Sequential()

La première couche convertit les représentations d'entiers en incrustations de vecteurs denses. Voir le [tutoriel sur les encastrements de mots](../text/word_embeddings.ipynb) ou plus de détails. 

In [0]:
# La couche Embedding permet de coder chaque élément du vocabulaire (valeur entière)
# avec 64 réels de 0 à 1 qui sont comme des poids qui seront déterminés lors de l'apprentissage
model.add(tf.keras.layers.Embedding(vocab_size, 64))

La couche suivante est une couche [Long Short-Term Memory] (http://colah.github.io/posts/2015-08-Understanding-LSTMs/), qui permet au modèle de comprendre les mots dans leur contexte avec d'autres mots. Un wrapper bidirectionnel sur le LSTM lui permet de connaître les points de données en relation avec les points de données qui l'ont précédé et suivi.

In [0]:
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

Enfin, nous aurons une série d'une ou plusieurs couches densément connectées, la dernière étant la couche de sortie. La couche de sortie produit une probabilité pour tous les labels. Celui qui a la plus forte probabilité est la prédiction des modèles du label d'un exemple.

In [0]:
# Une ou plusieurs couches denses.
# Editer la liste à la ligne `for` pour tester les tailles des couches.
for units in [64, 64]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

# Couche de sortie. L'argume,nt est le nombre de labels.
model.add(tf.keras.layers.Dense(3))

Enfin, compilez le modèle. Pour un modèle de catégorisation softmax, utilisez `sparse_categorical_crossentropy` comme fonction de perte. Vous pouvez essayer d'autres optimiseurs, mais `adam` est très courant.

In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Former le modèle

Ce modèle fonctionnant sur ces données donne des résultats décents (environ 83% avec 3 epochs). Nous allons améliorer ce résultat en passant à 10 epochs

In [0]:
model.fit(train_data, epochs=10, validation_data=test_data)

In [0]:
eval_loss, eval_acc = model.evaluate(test_data)

print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}'.format(eval_loss, eval_acc))